In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 6.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from transformers import ElectraTokenizer, TFElectraForSequenceClassification

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/EDA.csv")

In [ ]:
df.isnull().sum()

Utterance    11
Target        0
dtype: int64

In [ ]:
df= df.dropna()

In [ ]:
texts = df["Utterance"].values
labels = df["Target"].values

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow==<previous version>

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `pip install tensorflow==<previous version>'


In [ ]:
import tensorflow as tf
from transformers import TFElectraForSequenceClassification, ElectraTokenizer
from sklearn.preprocessing import LabelEncoder


def get_model(num_classes=7):
      # ELECTRA 모델 불러오기
      model = TFElectraForSequenceClassification.from_pretrained("google/electra-base-discriminator", num_labels=num_classes)
      return model

def preprocess(texts, tokenizer, max_seq_len=64):
    input_ids = []
    attention_mask = []

    for text in texts:
        encoding = tokenizer.encode_plus(text, add_special_tokens=True, max_length=max_seq_len, truncation=True)
        input_ids.append(encoding['input_ids'])
        attention_mask.append(encoding['attention_mask'])

    input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=max_seq_len, dtype='int32', padding='post', truncating='post')
    attention_mask = tf.keras.preprocessing.sequence.pad_sequences(attention_mask, maxlen=max_seq_len, dtype='int32', padding='post', truncating='post')

    input_ids = tf.convert_to_tensor(input_ids, dtype=tf.int32)
    attention_mask = tf.convert_to_tensor(attention_mask, dtype=tf.int32)

    return input_ids, attention_mask

def encode_labels(labels):
    encoder = LabelEncoder()
    encoder.fit(labels)
    return encoder.transform(labels)

In [ ]:
from tensorflow import keras
import os

# 모델을 저장해주는 콜백
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/khuda_team_project/models/electra_03.h5",
                                                      save_weights_only=True, 
                                                      save_best_only=True)
# 로그디렉토리를 편하게 보기위해 이름을 바꾸기 위한 코드
callbacks = [model_checkpoint_cb]

In [ ]:
X = df['Utterance'].values
y = df['Target'].values

# 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("google/electra-base-discriminator")

# 입력 데이터 전처리
X = preprocess(X, tokenizer, 64)
y = encode_labels(y)

In [ ]:
model = get_model(num_classes=7)

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Some layers from the model checkpoint at google/electra-base-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit([X[0],X[1]],
          y,
          epochs=8,
          batch_size=64,
          validation_split=0.1,
          callbacks=callbacks)

Epoch 1/8
453/453 [==============================] - 412s 809ms/step - loss: 1.1175 - accuracy: 0.6111 - val_loss: 1.5330 - val_accuracy: 0.6194
Epoch 2/8
453/453 [==============================] - 368s 811ms/step - loss: 0.6187 - accuracy: 0.7901 - val_loss: 1.7294 - val_accuracy: 0.5648
Epoch 3/8
453/453 [==============================] - 368s 812ms/step - loss: 0.5024 - accuracy: 0.8278 - val_loss: 1.6306 - val_accuracy: 0.6384
Epoch 4/8
453/453 [==============================] - 368s 811ms/step - loss: 0.4290 - accuracy: 0.8511 - val_loss: 1.8567 - val_accuracy: 0.6390
Epoch 5/8
453/453 [==============================] - 368s 811ms/step - loss: 0.3616 - accuracy: 0.8744 - val_loss: 2.0048 - val_accuracy: 0.6415
Epoch 6/8
453/453 [==============================] - 367s 811ms/step - loss: 0.2995 - accuracy: 0.8981 - val_loss: 2.2508 - val_accuracy: 0.6092
Epoch 7/8
453/453 [==============================] - 368s 812ms/step - loss: 0.2515 - accuracy: 0.9132 - val_loss: 2.2853 - val_ac

In [ ]:
model.save_weights('/content/drive/MyDrive/khuda_team_project/models/electra_04.h5')